In [2]:
from clipper_admin import ClipperConnection, DockerContainerManager
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()

20-03-10:05:08:26 INFO     [docker_container_manager.py:184] [default-cluster] Starting managed Redis instance in Docker
20-03-10:05:08:30 INFO     [docker_container_manager.py:276] [default-cluster] Metric Configuration Saved at /tmp/tmp3xh8049g.yml
20-03-10:05:08:31 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running


In [ ]:
# example from github
def feature_sum(xs):    
    return [str(sum(x)) for x in xs]
#a = [[1., 2., 3.]]
#b = feature_sum(a)
#print(b)#['6.0']

In [3]:
#插入排序
def insert_sort(lists):
#     list = lists[0]
    list1 = []
    for l in range(len(lists)):    
        list = lists[l]
        length = len(list)
        for i in range(1,length):
            for j in range(i):
                if list[j] > list[j+1]:
                    list[j],list[j+1] = list[j+1],list[j]
        list1.append(str(list))
    return list1
a = [[1,7,3]]
b = insert_sort(a)
print(b) 

['[1, 3, 7]']


In [53]:
def bubble_sort(lists):
    res = list()
    for l in range(len(lists)):    
        alist = lists[l]
        for j in range(len(alist)-1,0,-1):
            for i in range(j):
                if alist[i] > alist[i+1]:
                    alist[i], alist[i+1] = alist[i+1], alist[i]
        res.append(str(alist))
    return res
# a = [[1,7,3]]
# b = bubble_sort(a)
# print(b) 
# ['[1, 3, 7]']

In [12]:
def quick_sort(list6):
    list5 = []
#     for l in range(len(list6)): 
    lists = list6[0]
    print(lists)
    if lists == []:
        return []
    else:
        first = lists[0]
        left = quick_sort([l for l in lists[1:]if l < first])
        right = quick_sort([l for l in lists[1:] if l > first])
        list1 = left +[first] + right
    for i in list1:
        list5.append(str(list1))
    return list5
a = [[1,7,3]]
print(quick_sort(a))

[1, 7, 3]


IndexError: list index out of range

In [4]:
from clipper_admin.deployers import python as python_deployer

In [5]:
python_deployer.deploy_python_closure(clipper_conn, name="sum-model", version=1, input_type="int", func=insert_sort)

20-03-10:05:16:38 INFO     [deployer_utils.py:41] Saving function to /tmp/tmp_ax_i3vvclipper
20-03-10:05:16:38 INFO     [deployer_utils.py:51] Serialized and supplied predict function
20-03-10:05:16:38 INFO     [python.py:192] Python closure saved
20-03-10:05:16:38 INFO     [python.py:206] Using Python 3.6 base image
20-03-10:05:16:38 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /tmp/tmp_ax_i3vvclipper
20-03-10:05:16:38 INFO     [clipper_admin.py:539] [default-cluster] Step 1/2 : FROM clipper/python36-closure-container:0.4.1
20-03-10:05:16:38 INFO     [clipper_admin.py:539] [default-cluster]  ---> e5b9dc250c05
20-03-10:05:16:38 INFO     [clipper_admin.py:539] [default-cluster] Step 2/2 : COPY /tmp/tmp_ax_i3vvclipper /model/
20-03-10:05:16:38 INFO     [clipper_admin.py:539] [default-cluster]  ---> f1f806202ae8
20-03-10:05:16:38 INFO     [clipper_admin.py:539] [default-cluster] Successfully built f1f806202ae8
20-03-10:05:16:38 INFO   

In [6]:
clipper_conn.register_application(name="hello-world", input_type="int", default_output="-1.0", slo_micros=100000)

20-03-10:05:21:47 INFO     [clipper_admin.py:236] [default-cluster] Application hello-world was successfully registered


In [7]:
clipper_conn.link_model_to_app(app_name="hello-world", model_name="sum-model")

20-03-10:05:22:30 INFO     [clipper_admin.py:303] [default-cluster] Model sum-model is now linked to application hello-world


In [47]:
clipper_conn.get_all_apps()

['hello-world']

In [9]:
clipper_conn.get_all_models()

['sum-model:1']

In [7]:
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://localhost:1337/hello-world/predict", headers=headers, data=json.dumps({"input": [9,2,3]})).json()

{'query_id': 0, 'output': '[2 3 9]', 'default': False}

In [8]:
def predict(addr, x, batch=False):
    url = "http://%s/hello-world/predict" % addr

    if batch:
        req_json = json.dumps({'input_batch': x})
    else:
        req_json = json.dumps({'input': list(x)})

    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))

In [9]:
import json ,requests
from datetime import datetime
addr = "127.0.0.1:1337"
# x = '{"input": [7,2,3]}'
x = [1,8,2]
predict(addr,x,batch=False)

'{"query_id":0,"output":"[1 2 8]","default":false}', 11.235000 ms


In [ ]:
clipper_conn.delete_application(name="hello-world")

In [ ]:
clipper_conn.unlink_model_from_app(app_name="hello-world", model_name="sum-model")

In [10]:
clipper_conn.stop_all()

20-03-10:06:00:20 INFO     [clipper_admin.py:1424] [default-cluster] Stopped all Clipper cluster and all model containers


1，快排实现中用到递归 ，但是取元素会报超出索引 。暂没想到 好方法实现
2，输入batch可以为1，但是输入必须为双层数据。输出也是